# Create resaidual norm files for CREDIT

In [1]:
import os
import yaml
import copy
import numpy as np
import xarray as xr

In [2]:
from scipy.stats import gmean

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

## File creation

### 6 hourly residual coeff

In [4]:
# get variable information from data_preprocessing/config
config_name = os.path.realpath('data_config_6h.yml')

with open(config_name, 'r') as stream:
    conf = yaml.safe_load(stream)

In [5]:
N_levels = 37
base_dir = '/glade/derecho/scratch/ksha/CREDIT_data/ERA5_plevel_1deg/'

In [6]:
# get variable names
varnames = list(conf['residual'].keys())
varnames = varnames[:-5] # remove save_loc and others

varname_surf = list(set(varnames) - set(['U', 'V', 'T', 'Q', 'Z', 'specific_total_water']))
varname_upper = ['U', 'V', 'T', 'Q', 'Z', 'specific_total_water']

In [7]:
# collect computed mean and variance values
# See "qsub_STEP01_compute_mean_std.ipynb"
MEAN_values = {}
STD_values = {}

for varname in varname_surf:
    save_name = conf['residual']['save_loc'] + '{}_mean_std_{}.npy'.format(conf['residual']['prefix'], varname)
    mean_std = np.load(save_name)
    MEAN_values[varname] = mean_std[0]
    STD_values[varname] = mean_std[1]

for varname in varname_upper:

    # -------------------------------------------- #
    # allocate all levels
    mean_std_all_levels = np.empty((2, N_levels))
    mean_std_all_levels[...] = np.nan
    
    for i_level in range(N_levels):
        save_name = conf['residual']['save_loc'] + '{}_level{}_mean_std_{}.npy'.format(conf['residual']['prefix'], i_level, varname)
        mean_std = np.load(save_name)
        mean_std_all_levels[:, i_level] = mean_std

    # -------------------------------------------- #
    # save
    MEAN_values[varname] = np.copy(mean_std_all_levels[0, :])
    STD_values[varname] = np.copy(mean_std_all_levels[1, :])

keys_to_drop = ['VAR_10U', 'VAR_10V', 'SP', 'Q', 'Q500', 'land_sea_CI_mask']
MEAN_values = {k: v for k, v in MEAN_values.items() if k not in keys_to_drop}
STD_values = {k: v for k, v in STD_values.items() if k not in keys_to_drop}

In [8]:
std_val_all = list(STD_values.values())
std_val_surf = np.array(std_val_all[:-5])
std_val_upper = std_val_all[-5:]

In [9]:
std_concat = np.concatenate([std_val_surf]+ std_val_upper)
std_g = gmean(np.sqrt(std_concat))

In [10]:
ds_example = xr.open_zarr(base_dir+'all_in_one/ERA5_plevel_1deg_6h_1979_conserve.zarr')
level = np.array(ds_example['level'])

In [11]:
# ------------------------------------------------------- #
# create xr.DataArray for std
ds_std_6h = xr.Dataset(coords={"level": level})

for varname, data in STD_values.items():
    data = np.sqrt(data) / std_g # <--- var to std and divided by std_g
    if len(data.shape) == 1:
        data_array = xr.DataArray(
            data,
            dims=["level",],
            coords={"level": level},
            name=varname,
        )
        ds_std_6h[varname] = data_array
    else:
        data_array = xr.DataArray(
            data,
            name=varname,
        )
        ds_std_6h[varname] = data_array

In [12]:
ds_std_6h.to_netcdf(base_dir+'mean_std/residual_6h_1979_2019_conserve_1deg.nc')

In [14]:
# ------------------------------------------------------- #
# Compare with my old ones
std_conserve = xr.open_dataset(base_dir+'mean_std/residual_6h_1979_2019_conserve_1deg.nc')
std_bilinear = xr.open_dataset(base_dir+'mean_std/residual_6h_1979_2019_bilinear_1deg.nc')

for varname in list(std_conserve.keys()):
    print('=============== {} ================='.format(varname))
    print(np.array(std_conserve[varname]))
    print(np.array(std_bilinear[varname]))

=============== SKT =================
0.9020328939906355
0.8889482229414569
=============== surface_latent_heat_flux =================
2.756990226781677
2.7338782622675653
=============== top_net_solar_radiation =================
6.285933663515951
6.097818137560312
=============== VAR_2T =================
0.5812322200302392
0.5771408616318606
=============== surface_net_thermal_radiation =================
3.1111771870427525
3.082331817881059
=============== toa_incident_solar_radiation =================
6.302868904142561
6.115183631423283
=============== surface_sensible_heat_flux =================
5.077296641030351
4.921228881376689
=============== MSL =================
0.9710019153823931
0.947887088931082
=============== TCC =================
3.451359736171103
3.6788048004145764
=============== total_precipitation =================
4.418252724674436
4.625040749189226
=============== top_net_thermal_radiation =================
1.8856349420386707
1.9081347668682982
=============== evap

In [21]:
# ------------------------------------------------------- #
# Compare with my old ones
new_std = xr.open_dataset('/glade/derecho/scratch/ksha/CREDIT_data/ERA5_plevel_1deg/mean_std/residual_6h_1979_2019_full_1deg.nc')

for varname in list(new_std.keys()):
    print('=============== {} ================='.format(varname))
    print(np.array(new_std[varname]))

=============== evaporation =================
2.624155534902845
=============== surface_net_thermal_radiation =================
2.963156138014652
=============== top_net_solar_radiation =================
5.862051300897985
=============== total_precipitation =================
4.44621756977132
=============== SKT =================
0.85457781277974
=============== toa_incident_solar_radiation =================
5.878745373038826
=============== surface_latent_heat_flux =================
2.628175236172955
=============== top_net_thermal_radiation =================
1.8343583950970768
=============== VAR_2T =================
0.5548262120004838
=============== TCC =================
3.536567011270057
=============== MSL =================
0.9112378587590986
=============== surface_net_solar_radiation =================
5.825752161603732
=============== surface_sensible_heat_flux =================
4.730953845342623
=============== U =================
[1.03259926 0.9297491  0.85987739 0.76928461 0.